In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [12]:
searchKeyword='기념품' # 전체의 기준이 되는 부분
gov='seoul'
startDate='2016-01-01'
endDate='2022-05-16'
items_per_page='50'

In [13]:
url= f'https://opengov.seoul.go.kr/sanction/list?items_per_page={items_per_page}&gov={gov}&searchKeyword={searchKeyword}&rangeDate=custom&startDate={startDate}&endDate={endDate}'
print(url)

https://opengov.seoul.go.kr/sanction/list?items_per_page=50&gov=seoul&searchKeyword=기념품&rangeDate=custom&startDate=2016-01-01&endDate=2022-05-16


In [14]:
def get_count(url):
    response=requests.get(url)
    html= response.text
    soup= BeautifulSoup(html, 'lxml')
    count=soup.select('div[class=al-left]')[0].text
    p= re.compile('\d+,*\d*')
    page_count=(int(p.search(count).group().replace(',',''))//int(items_per_page))+1
    return page_count

In [15]:
get_count(url)

79

In [16]:
DFlist=[]
for i in range(get_count(url)):
    DFlist.append('DF'+str(i))

In [17]:
len(DFlist)

79

In [18]:
def get_data(page_input):
    response= requests.get(f'{url}&page={page_input}')
    print(f'{url}&page={page_input}')
    html = response.text
    soup= BeautifulSoup(html, 'lxml')
    title_list=[]
    public_list=[]
    govern_list=[]
    URL_list=[]
    dept_list=[]
    date_list=[]
    for tag in soup.select('div[class=title-area]'):
        title_list.append(tag.find('a').text.replace('제목 :','').strip())
    for tag in soup.select('p[class=title-category]'):
        public_list.append(tag.findAll('span')[0].text)
        govern_list.append(tag.findAll('span')[1].text)
    for tag in soup.select('p[class=title-info]'):
        date_list.append(tag.find('span', attrs={'class':'date'}).text.replace('등록일 :','').strip())
    for tag in soup.select('p[class=title-info]'):
        dept_list.append(tag.find('span', attrs={'class':'dept'}).text.replace('부서 :','').strip())
    for tag in soup.select('div[class=title-area]'):
        URL_list.append('https://opengov.seoul.go.kr'+tag.find('a')['href'])
    data= {'title': title_list, 'URL':URL_list, 'date': date_list, 'dept': dept_list, 'IsPublic': public_list, 'gov':govern_list}
    DFlist[page_input-1]= pd.DataFrame(data=data)
    return DFlist[page_input-1]

In [19]:
for i in range(1,get_count(url)+1):
    get_data(i)

https://opengov.seoul.go.kr/sanction/list?items_per_page=50&gov=seoul&searchKeyword=기념품&rangeDate=custom&startDate=2016-01-01&endDate=2022-05-16&page=1
https://opengov.seoul.go.kr/sanction/list?items_per_page=50&gov=seoul&searchKeyword=기념품&rangeDate=custom&startDate=2016-01-01&endDate=2022-05-16&page=2
https://opengov.seoul.go.kr/sanction/list?items_per_page=50&gov=seoul&searchKeyword=기념품&rangeDate=custom&startDate=2016-01-01&endDate=2022-05-16&page=3
https://opengov.seoul.go.kr/sanction/list?items_per_page=50&gov=seoul&searchKeyword=기념품&rangeDate=custom&startDate=2016-01-01&endDate=2022-05-16&page=4
https://opengov.seoul.go.kr/sanction/list?items_per_page=50&gov=seoul&searchKeyword=기념품&rangeDate=custom&startDate=2016-01-01&endDate=2022-05-16&page=5
https://opengov.seoul.go.kr/sanction/list?items_per_page=50&gov=seoul&searchKeyword=기념품&rangeDate=custom&startDate=2016-01-01&endDate=2022-05-16&page=6
https://opengov.seoul.go.kr/sanction/list?items_per_page=50&gov=seoul&searchKeyword=기념품&

In [20]:
total_DF= pd.concat([i for i in DFlist], ignore_index=True)

In [21]:
len(DFlist)

79

In [22]:
total_DF

,title,URL,date,dept,IsPublic,gov
0,기념품 배부 결과보고(UV살균 소독기),https://opengov.seoul.go.kr/sanction/25976228,2022-05-16,송파소방서 소방행정과,부분공개,서울시
1,국제 교류 협력 증진을 위한 전통기념품 구매 비용 지급,https://opengov.seoul.go.kr/sanction/25977766,2022-05-16,의회사무처 의정담당관,공개,서울시
2,국제 교류 협력 증진을 위한 전통기념품 구매 비용 지급,https://opengov.seoul.go.kr/sanction/25975944,2022-05-16,의회사무처 의정담당관,공개,서울시
3,관악소방서 내방객 답례용 기념품 구매 계획 보고(소화기),https://opengov.seoul.go.kr/sanction/25970773,2022-05-13,관악소방서 소방행정과,부분공개,서울시
4,산하공원 프로그램 기념품 구매,https://opengov.seoul.go.kr/sanction/25952534,2022-05-11,동부공원녹지사업소 공원여가과,부분공개,서울시
...,...,...,...,...,...,...
3915,서울 상징 관광기념품 전시회 개최 용역비 지급,https://opengov.seoul.go.kr/sanction/7205721,2016-01-08,관광체육국 관광사업과,부분공개,서울시
3916,2015년 하반기 성과평가 최우수상 수상 기념품 등 배정 계획,https://opengov.seoul.go.kr/sanction/7207895,2016-01-08,관악소방서 소방행정과,공개,서울시
3917,‘15 서울 상징 관광기념품 공모전 수상작 전시회 개최결과 보고,https://opengov.seoul.go.kr/sanction/7166162,2016-01-05,관광체육국 관광사업과,공개,서울시
3918,서울 상징 관광기념품 지원 요청,https://opengov.seoul.go.kr/sanction/7157589,2016-01-04,기획조정실 국제교류담당관,공개,서울시


In [23]:
fname= f'{searchKeyword}_{gov}_{startDate}_{endDate}.xlsx'
total_DF.to_excel(fname, index=False)